In [1]:
import numpy as np 
import pandas as pd 

In [2]:
trn_id = pd.read_csv("/home/sid2018-1/Bureau/kaggle_fraud/data/train_identity.csv")
trn_tr = pd.read_csv("/home/sid2018-1/Bureau/kaggle_fraud/data/train_transaction.csv")
test_id= pd.read_csv("/home/sid2018-1/Bureau/kaggle_fraud/data/test_identity.csv")
test_tr= pd.read_csv("/home/sid2018-1/Bureau/kaggle_fraud/data/test_transaction.csv")

In [3]:
train = pd.merge(trn_id, trn_tr, on = 'TransactionID', how = 'right')
test = pd.merge(test_id, test_tr, on = 'TransactionID', how = 'right')

In [4]:
cols = 'C10, C11,C12 ,C13, C14, C2, C4, C5, C6, C7, C8, C9, D11, D2, D4, D6, D7, M1 ,M2, M3, M5, M6, M7, M8, M9, V1, V10, V102, V103,V105, V106, V109, V11, V110, V112, V113, V115, V116, V118, V119, V121, V122, V125, V126, V127, V128, V132, V133, V134, V137,V140, V143, V145, V147, V149, V150, V151, V152, V153, V154, V155, V156, V157, V158, V159, V16, V160, V163,V164, V167, V168 ,V177, V178, V179, V18, V182, V183, V190, V192, V193, V196, V197, V198, V199, V2, V20, V201, V202, V203, V204, V206, V207, V211,V212 ,V213, V216, V217, V218, V219, V222, V225, V231 ,V232, V233, V235, V236, V237, V239, V24, V243, V245, V249, V251, V253, V254,V256 ,V257, V259, V263, V265, V266, V269, V271, V272, V273, V274, V275, V276, V277, V278, V279, V28, V280, V287, V289, V292, V293, V294,V295 ,V296, V297, V298, V299, V3, V301, V306, V307, V308, V309, V310, V311, V312, V315, V316, V317, V318, V32, V321, V322, V323, V324,V325 ,V326, V327, V328, V329, V33, V330, V331, V332, V333, V334, V336, V339, V34, V4, V41, V45, V5, V6, V65, V68, V7, V74, V8, V88, V89, V9,V94, V95, V96, V97, V98, V99'

In [5]:
cols_to_delete = cols.replace(' ','').split(",")

In [86]:
train1 = train.drop(cols_to_delete, axis = 1)
test1 = test.drop(cols_to_delete, axis = 1)

In [87]:
test1.shape

(506691, 239)

In [88]:
listenan = ['addr2','C3','C5','C9','D10', 'D11', 'D12','D13','D14', 'D3', 'D5', 'D8','D9','DeviceInfo','dist1','dist2','id_03','id_04','id_09','id_07','id_08','id_10','id_14','id_18','id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_30','id_32','id_33','id_34','M1','M2','M3','M5','M6','M7','M8','M9','V1','V10','V100','V101','V104','V11','V12','V13','V138','V139','V14','V141','V142','V144','V146','V148','V15','V161','V162','V165','V166','V169','V17','V19','V2','V21','V22','V23','V25','V255','V26','V27','V29','V3','V30','V31','V335','V337','V338','V35','V36','V37','V38','V39','V40','V42','V43','V44','V50','V46','V47','V48','V49','V5','V51','V52','V53','V54','V55','V56','V57','V58','V59','V60','V6','V61','V62','V63','V64','V70','V66','V67','V71','V69','V7','V72','V73','V80','V75','V76','V77','V78','V79','V8','V81','V82','V83','V84','V85','V86','V87','V9','V90','V91','V92','V93']

In [89]:
for x in listenan:
     if x in train1.columns:
            train1=  train1.drop([x], axis = 1)

In [90]:
for x in listenan:
     if x in test1.columns:
            test1=  test1.drop([x], axis = 1)

In [91]:
todel= ['card6','DeviceType','V107','V305']

In [92]:
train1 = train1.drop(todel, axis=1)

In [93]:
test1 = test1.drop(todel, axis=1)

In [94]:
## Some more features 

In [95]:
# New feature - decimal part of the transaction amount
train1['TransactionAmt_decimal'] = ((train1['TransactionAmt'] - train1['TransactionAmt'].astype(int)) * 1000).astype(int)
test1['TransactionAmt_decimal'] = ((test1['TransactionAmt'] - test1['TransactionAmt'].astype(int)) * 1000).astype(int)

# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
train1['card1_count_full'] = train1['card1'].map(pd.concat([train1['card1'], test1['card1']], ignore_index=True).value_counts(dropna=False))
test1['card1_count_full'] = test1['card1'].map(pd.concat([train1['card1'], test1['card1']], ignore_index=True).value_counts(dropna=False))

# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
train1['Transaction_day_of_week'] = np.floor((train1['TransactionDT'] / (3600 * 24) - 1) % 7)
test1['Transaction_day_of_week'] = np.floor((test1['TransactionDT'] / (3600 * 24) - 1) % 7)
train1['Transaction_hour'] = np.floor(train1['TransactionDT'] / 3600) % 24
test1['Transaction_hour'] = np.floor(test1['TransactionDT'] / 3600) % 24

In [96]:
train1['TransactionAmt_to_mean_card1'] = train1['TransactionAmt'] / train1.groupby(['card1'])['TransactionAmt'].transform('mean')
train1['TransactionAmt_to_mean_card4'] = train1['TransactionAmt'] / train1.groupby(['card4'])['TransactionAmt'].transform('mean')
train1['TransactionAmt_to_std_card1'] = train1['TransactionAmt'] / train1.groupby(['card1'])['TransactionAmt'].transform('std')
train1['TransactionAmt_to_std_card4'] = train1['TransactionAmt'] / train1.groupby(['card4'])['TransactionAmt'].transform('std')

test1['TransactionAmt_to_mean_card1'] = test1['TransactionAmt'] / test1.groupby(['card1'])['TransactionAmt'].transform('mean')
test1['TransactionAmt_to_mean_card4'] = test1['TransactionAmt'] / test1.groupby(['card4'])['TransactionAmt'].transform('mean')
test1['TransactionAmt_to_std_card1'] = test1['TransactionAmt'] / test1.groupby(['card1'])['TransactionAmt'].transform('std')
test1['TransactionAmt_to_std_card4'] = test1['TransactionAmt'] / test1.groupby(['card4'])['TransactionAmt'].transform('std')


In [97]:
train1['id_02_to_mean_card1'] = train1['id_02'] / train1.groupby(['card1'])['id_02'].transform('mean')
train1['id_02_to_mean_card4'] = train1['id_02'] / train1.groupby(['card4'])['id_02'].transform('mean')
train1['id_02_to_std_card1'] = train1['id_02'] / train1.groupby(['card1'])['id_02'].transform('std')
train1['id_02_to_std_card4'] = train1['id_02'] / train1.groupby(['card4'])['id_02'].transform('std')

test1['id_02_to_mean_card1'] = test1['id_02'] / test1.groupby(['card1'])['id_02'].transform('mean')
test1['id_02_to_mean_card4'] = test1['id_02'] / test1.groupby(['card4'])['id_02'].transform('mean')
test1['id_02_to_std_card1'] = test1['id_02'] / test1.groupby(['card1'])['id_02'].transform('std')
test1['id_02_to_std_card4'] = test1['id_02'] / test1.groupby(['card4'])['id_02'].transform('std')

train1['D15_to_mean_card1'] = train1['D15'] / train1.groupby(['card1'])['D15'].transform('mean')
train1['D15_to_mean_card4'] = train1['D15'] / train1.groupby(['card4'])['D15'].transform('mean')
train1['D15_to_std_card1'] = train1['D15'] / train1.groupby(['card1'])['D15'].transform('std')
train1['D15_to_std_card4'] = train1['D15'] / train1.groupby(['card4'])['D15'].transform('std')

In [98]:
test1['D15_to_mean_card1'] = test1['D15'] / test1.groupby(['card1'])['D15'].transform('mean')
test1['D15_to_mean_card4'] = test1['D15'] / test1.groupby(['card4'])['D15'].transform('mean')
test1['D15_to_std_card1'] = test1['D15'] / test1.groupby(['card1'])['D15'].transform('std')
test1['D15_to_std_card4'] = test1['D15'] / test1.groupby(['card4'])['D15'].transform('std')

In [99]:
train1[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train1['P_emaildomain'].str.split('.', expand=True)
train1[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train1['R_emaildomain'].str.split('.', expand=True)
test1[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test1['P_emaildomain'].str.split('.', expand=True)
test1[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test1['R_emaildomain'].str.split('.', expand=True)

In [100]:
## Binarize transaction Amount to create foreign feature 
train1['foreign'] = [0 if (x - x.astype(int)== 0) else -1 for x in train1['TransactionAmt'].values]
test1['foreign'] = [0 if (x - x.astype(int)== 0) else -1 for x in test1['TransactionAmt'].values]

In [102]:
listcols = ['id_31','R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3','P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3','P_emaildomain', 'R_emaildomain','V120','V117','ProductCD','id_38','id_35','id_36','id_37','id_12','id_15','id_16','id_28','id_29','card4']

In [22]:
from sklearn import preprocessing
def tranformcols(listcols, train, test):
    for col in listcols:
        le = preprocessing.LabelEncoder()
        le.fit([str(x) for x in train[col].values] +[str(x) for x in test[col].values] )
        test[col] = le.transform([str(x) for x in test[col].values])
        train[col] = le.transform([str(x) for x in train[col].values])
    return train,test

In [23]:
train1,test1= tranformcols(listcols, train1, test1)

In [24]:
navig = ['chrome','firefox', 'samsung', 'google', 'ie', 'edge', 'opera', 'safari']

In [25]:
def lab_navig(navig,df):
    label =[]
    for k,val in enumerate (df['id_31'].values):
        ik = False
        val = str(val).lower()
        for i,j in enumerate (navig):
            if ik == False:
                if j in val:
                    label.append(i)
                    ik = True
        if len(label)== k:
            label.append(8)
    df['id_31'] = label
    return df

In [26]:
train1 = lab_navig(navig,train1)
test1 = lab_navig(navig,test1)

In [103]:
train1 = train1.set_index(train1['TransactionID'])
test1 = test1.set_index(test1['TransactionID'])

In [104]:
train1 = train1.drop(['TransactionID', 'TransactionDT'], axis = 1)
test1 = test1.drop(['TransactionID', 'TransactionDT'], axis = 1)

In [105]:
train1 = train1.drop(['M4'], axis = 1)
test1 = test1.drop(['M4'], axis = 1)

 replace Nan using knn

In [106]:
train1 = train1.replace([np.inf, -np.inf], np.nan)
test1 = test1.replace([np.inf, -np.inf], np.nan)


In [31]:
train2 = train1
test2 = test1

In [117]:
train2 = train2.fillna(train2.median())
test2 = test2.fillna(test2.median())

In [122]:
train2 = train2.drop(['isolation'], axis = 1)
test2 = test2.drop(['isolation'], axis = 1)

In [109]:
ypred1 = train1['isFraud'].values

In [110]:
train1 = train1.drop(['isFraud'], axis = 1)

In [35]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(train1, ypred1, test_size=0.2, random_state=42)

In [36]:
train1.shape, test1.shape

((590540, 137), (506691, 137))

In [112]:
train1['isolation']= train2['isolation']
test1['isolation']= test2['isolation']

##### isolation forest as label 

In [37]:
from sklearn.ensemble import IsolationForest
# training the model
rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=100, random_state=rng)
clf.fit(train1)

# predictions
y_pred_train = clf.predict(train1)
y_pred_test = clf.predict(test1)

/home/sid2018-1/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/home/sid2018-1/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/sid2018-1/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/sid2018-1/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [38]:
train1["isolation"] = y_pred_train
test1["isolation"] = y_pred_test

In [143]:
##### DBscan

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=3, min_samples=2).fit(train1)
clustering1 = DBSCAN(eps=3, min_samples=2).fit(test1)
train1["dbscan"] = clustering.labels_
test1["dbscan"] = clustering1.labels_


In [39]:
### NLP features 

In [40]:
import flair as fl

In [42]:
nlptr = train[['card4','card6', 'id_31','P_emaildomain', 'R_emaildomain','isFraud']]
nlptst = test[['card4','card6', 'id_31','P_emaildomain', 'R_emaildomain']]
nlptst= nlptst.set_index(test['TransactionID'])
nlptr= nlptr.set_index(train['TransactionID'])

In [43]:
def flairize(df, tpe):
    l = []
    for x in df.index:
        v = df.loc[x].values
        l.append('card4_'+str(v[0])+' card6_'+str(v[1])+' navig_'+str(v[2])+' ped_'+str(v[3])+' red_'+str(v[4]))
    df['text'] = l
    if tpe == 'train':
        df['isFraud'] = ['__label__'+str(x) for x in df['isFraud'].values]
    return (df)

In [44]:
nlptr = flairize(nlptr,'train')[['isFraud','text']]
nlptr['flair']= [nlptr.loc[x].values[0]+' '+nlptr.loc[x].values[1] for x in nlptr.index ]
nlptr = nlptr[['flair']]

In [45]:
nlptst = flairize(nlptst,'test')[['text']]

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
tr, tst = train_test_split(nlptr, test_size=0.2)

In [46]:
np.savetxt('/home/sid2018-1/Bureau/train.txt',nlptr['flair'].values, delimiter=',', fmt='%s')
np.savetxt('/home/sid2018-1/Bureau/test.txt',nlptst['text'].values, delimiter=',', fmt='%s')

In [55]:
#! wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
#!unzip v0.1.0.zip


Archive:  v0.1.0.zip
431c9e2a9b5149369cc60fb9f5beba58dcf8ca17
   creating: fastText-0.1.0/
  inflating: fastText-0.1.0/.gitignore  
  inflating: fastText-0.1.0/CONTRIBUTING.md  
  inflating: fastText-0.1.0/LICENSE  
  inflating: fastText-0.1.0/Makefile  
  inflating: fastText-0.1.0/PATENTS  
  inflating: fastText-0.1.0/README.md  
  inflating: fastText-0.1.0/classification-example.sh  
  inflating: fastText-0.1.0/classification-results.sh  
  inflating: fastText-0.1.0/eval.py  
  inflating: fastText-0.1.0/get-wikimedia.sh  
  inflating: fastText-0.1.0/pretrained-vectors.md  
  inflating: fastText-0.1.0/quantization-example.sh  
  inflating: fastText-0.1.0/quantization-results.sh  
   creating: fastText-0.1.0/src/
  inflating: fastText-0.1.0/src/args.cc  
  inflating: fastText-0.1.0/src/args.h  
  inflating: fastText-0.1.0/src/dictionary.cc  
  inflating: fastText-0.1.0/src/dictionary.h  
  inflating: fastText-0.1.0/src/fasttext.cc  
  inflating: fastText-0.1.0/src/fasttext.h  
  inflat

In [57]:
#./fasttext supervised -input "/home/sid2018-1/Bureau/train.txt" -output "/home/sid2018-1/ftmodelt" -label __label__ -wordNgrams 2 -epoch 10
#! ./fasttext predict "/home/sid2018-1/ftmodelt.bin" "/home/sid2018-1/Bureau/test.txt" 1 > predtest1
  

/bin/sh: 1: ./fasttext: not found


In [47]:
# lecture du fichier predit
text_file = open("/home/sid2018-1/fastText-0.1.0/predtest1", "r")
lines = text_file.readlines()
text_file.close()

In [49]:
len(lines)

506691

## Random Forest Classifier

In [305]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(xtrain, ytrain) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [306]:
y_pred = [x[1] for x in clf.predict_proba(xtest)]

In [307]:
from sklearn import metrics
print(metrics.roc_auc_score(ytest,y_pred))

0.7714684831990408


In [308]:
y_pred

[0.025505998661621298,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025505998661621298,
 0.025764387548758948,
 0.027618231511035703,
 0.02640200945752104,
 0.03737907351481133,
 0.025505998661621298,
 0.025505998661621298,
 0.025764387548758948,
 0.06264888999936437,
 0.025764387548758948,
 0.03199717017347755,
 0.02640200945752104,
 0.025505998661621298,
 0.028168251626748297,
 0.02640200945752104,
 0.025505998661621298,
 0.02614362057038339,
 0.025764387548758948,
 0.025505998661621298,
 0.025764387548758948,
 0.04148949941859687,
 0.025764387548758948,
 0.028043725995508237,
 0.025764387548758948,
 0.025764387548758948,
 0.025505998661621298,
 0.025764387548758948,
 0.025505998661621298,
 0.025764387548758948,
 0.025764387548758948,
 0.04148949941859687,
 0.16444841579763492,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025764387548758948,
 0.025505998661621298

## XGBOOST

In [122]:
! pip install xgboost

    100% |████████████████████████████████| 142.8MB 485kB/s eta 0:00:011
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
phik 0.9.8 has requirement numpy>=1.15.4, but you'll have numpy 1.15.1 which is incompatible.
jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.9 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [50]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [51]:
# fit model no training data
model = XGBClassifier()
model.fit(train1, ypred1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [52]:
y_pred = model.predict_proba(test1)

In [53]:
y_pred = [x[1] for x in y_pred]

###### LGBM

In [58]:
! pip install setuptools wheel numpy scipy scikit-learn -U

Requirement already up-to-date: setuptools in ./anaconda3/lib/python3.7/site-packages (41.2.0)
    100% |████████████████████████████████| 20.3MB 2.1MB/s ta 0:00:011
    100% |████████████████████████████████| 25.2MB 2.3MB/s eta 0:00:01
Requirement already up-to-date: scikit-learn in ./anaconda3/lib/python3.7/site-packages (0.21.3)
Requirement not upgraded as not directly required: joblib>=0.11 in ./anaconda3/lib/python3.7/site-packages (from scikit-learn) (0.13.2)
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
thinc 6.12.0 has requirement dill<0.3.0,>=0.2.7, but you'll have dill 0.3.1.1 which is incompatible.
spacy 2.0.16 has requirement dill<0.3,>=0.2, but you'll have dill 0.3.1.1 which is incompatible.
jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: wheel 0.31.1
    Uninstalling wheel-0.31.1:
      Successfully uninstalled wheel-0.31.1
  Found existing i

In [61]:
! pip install lightgbm

    100% |████████████████████████████████| 1.3MB 26.2MB/s ta 0:00:01
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
thinc 6.12.0 has requirement dill<0.3.0,>=0.2.7, but you'll have dill 0.3.1.1 which is incompatible.
spacy 2.0.16 has requirement dill<0.3,>=0.2, but you'll have dill 0.3.1.1 which is incompatible.
jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.9 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [76]:
train2['isolation'] = train1['isolation']
test2['isolation'] = test1['isolation'] 

In [120]:
train1['isolation']

TransactionID
2987004    1
2987008    1
2987010   -1
2987011    1
2987016    1
2987017    1
2987022    1
2987038   -1
2987040   -1
2987048    1
2987049    1
2987057    1
2987066    1
2987069   -1
2987070    1
2987072    1
2987074    1
2987084    1
2987093    1
2987099   -1
2987100    1
2987101    1
2987104    1
2987105    1
2987108    1
2987111    1
2987114    1
2987119    1
2987121    1
2987125   -1
          ..
3577503    1
3577504    1
3577505    1
3577507    1
3577508    1
3577510    1
3577511    1
3577512    1
3577513    1
3577514    1
3577515    1
3577516    1
3577517    1
3577518    1
3577519    1
3577520    1
3577522    1
3577523    1
3577524    1
3577525    1
3577527    1
3577528    1
3577530    1
3577532    1
3577533    1
3577535    1
3577536    1
3577537    1
3577538    1
3577539    1
Name: isolation, Length: 590540, dtype: int64

In [77]:
import lightgbm as lgb

In [123]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
         }

In [124]:
d_train = lgb.Dataset(train2, label=ypred1)

In [125]:
clf = lgb.train(params, d_train, verbose_eval=False, num_boost_round = 1000)

In [127]:
predict2 = clf.predict(test2)

In [128]:
predict2

array([0.93186024, 0.88543759, 0.90448159, ..., 0.91407722, 0.89874496,
       0.92848663])

In [204]:
ypreds = [x[1] for x in predict]

In [139]:
d = {'TransactionID': test1.index, 'isFraud': y6}

In [140]:
df_soum = pd.DataFrame(data = d)

In [141]:
df_soum.to_csv('soum6', sep = ',', header = 'True', index= False)

In [ ]:
mean(xgboost, lightgbm)

In [75]:
xgb1 = y_pred
lgb1 = predict

In [126]:
xgb1

[0.028898394,
 0.04861356,
 0.02468041,
 0.038620874,
 0.3070343,
 0.023910983,
 0.03258159,
 0.03548591,
 0.05784229,
 0.009403653,
 0.091884084,
 0.02034172,
 0.022813689,
 0.030900318,
 0.024204142,
 0.024716303,
 0.035654604,
 0.02179576,
 0.049955636,
 0.058307406,
 0.026200943,
 0.010920339,
 0.06685374,
 0.036115892,
 0.02038532,
 0.024629267,
 0.50262684,
 0.101930544,
 0.12949355,
 0.16979054,
 0.02413519,
 0.19912815,
 0.030274803,
 0.0645645,
 0.016950196,
 0.15966082,
 0.18764363,
 0.2207357,
 0.36152095,
 0.05106205,
 0.925505,
 0.91759634,
 0.023025585,
 0.98004293,
 0.024944782,
 0.041648474,
 0.058215708,
 0.013680962,
 0.015100779,
 0.024758564,
 0.022915259,
 0.023390757,
 0.03206462,
 0.08494798,
 0.13940479,
 0.04471265,
 0.028675806,
 0.030895071,
 0.019208841,
 0.023085753,
 0.0443846,
 0.11243259,
 0.029440403,
 0.031376053,
 0.036246546,
 0.017047212,
 0.037218686,
 0.056036383,
 0.11425869,
 0.020223247,
 0.11425869,
 0.2109931,
 0.022352716,
 0.031091824,
 0.0

In [131]:
y5= ((0.9267/(0.9267+0.8822))* np.array(lgb1)) + ((0.8822/(0.9267+0.8822))* np.array(xgb1))

In [138]:
y6= ((0.9267/(0.9267))* np.array(lgb1)) + ((0.8822/(0.9267))* np.array(xgb1))

In [137]:
0.8822/(0.9267)

0.9519801445991152